In [ ]:
def create_parameter_row(idx, parameters, language='es'):
    translations = {
        "es": {
            "fields": ["difusión", "reproducción", "expansión", "pendiente", "caminos"],
            "help_texts": ["Valor del coeficiente de {}."]
        },
        "en": {
            "fields": ["diffusion", "reproduction", "expansion", "slope", "paths"],
            "help_texts": ["Value of the {} coefficient."]
        },
        "pt": {
            "fields": ["difusão", "reprodução", "expansão", "inclinação", "caminhos"],
            "help_texts": ["Valor do coeficiente de {}."]
        }
    }

    cols = []

    for field, name in zip(FIELDS, translations[language]["fields"]):
        col = dbc.Col(
            dbc.InputGroup(
                [
                    dbc.InputGroupText(
                        help_text(field.title(), translations[language]["help_texts"][0].format(name))
                    ),
                    dbc.Input(
                        type="number",
                        required=True,
                        value=parameters[field],
                        placeholder=1,
                        min=1,
                        max=100,
                        id={
                            "type": "val-prediction-coefficient",
                            "field": field,
                            "index": idx,
                        },
                        debounce=True,
                    ),
                ],
                class_name="mb-2",
            ),
            width=3,
        )
        cols.append(col)

    cols.append(
        dbc.Col(
            dbc.Button(
                "⨉", class_name="bg-danger", id={"type": "btn-delete-row", "index": idx}
            ),
            width=1,
            class_name="text-center",
        )
    )
    return dbc.Row(cols, class_name="mb-2", id={"type": "row-parameters", "index": idx})

In [ ]:
def summary(id_hash, urban_rasters, years, language='es'):
    translations = {
        "es": {
            "expansion": "Expansión",
            "observations": "Observaciones",
            "urbanization_percentage": "Porcentaje de urbanización",
            "year": "Año",
            "category": "Categoría",
            "general_summary": "Resumen General",
            "urban_area_increase": "{}% de aumento del área urbanizada 2070 vs. 2020."
        },
        "en": {
            "expansion": "Expansion",
            "observations": "Observations",
            "urbanization_percentage": "Urbanization Percentage",
            "year": "Year",
            "category": "Category",
            "general_summary": "General Summary",
            "urban_area_increase": "{}% increase in urban area 2070 vs. 2020."
        },
        "pt": {
            "expansion": "Expansão",
            "observations": "Observações",
            "urbanization_percentage": "Percentagem de Urbanização",
            "year": "Ano",
            "category": "Categoria",
            "general_summary": "Resumo Geral",
            "urban_area_increase": "{}% de aumento da área urbana 2070 vs. 2020."
        }
    }
    
    
    path_cache = Path(f"./data/cache/{id_hash}")

    worldcover = np.load(path_cache / "worldcover.npy")

    start_year = 2020
    num_years = 50

    modes = ["inercial", "acelerada", "controlada"]
    id_hash = str(id_hash)

    historical_years = np.array(years)
    historical_grids = np.array(urban_rasters)

    x = list(historical_years)
    y = [grid.sum() / grid.size for grid in historical_grids]
    z = ["Observaciones"] * len(x)

    final_y = y[-1]

    tabs = []
    coverage_graphs = []
    for mode in modes:
        grids = load_sleuth_predictions(path_cache, id_hash, mode=mode)

        x_pred = list(range(start_year + 1, start_year + num_years + 1))
        y_pred = [grid.sum() / grid.size for grid in grids]
        z_pred = [f"Expansión {mode}"] * (len(x_pred) + 1)

        x_pred = [start_year] + x_pred
        y_pred = [final_y] + y_pred

        x.extend(x_pred)
        y.extend(y_pred)
        z.extend(z_pred)

        # Plot Sleuth Predictions
        tab = dbc.Tab(
            dbc.Card(
                dbc.CardBody(
                    dbc.Container(
                        dbc.Row(
                            dbc.Col(
                                dcc.Graph(
                                    figure=plot_sleuth_predictions(grids, 2020, 50),
                                    responsive=True,
                                    style={"height": "60vh"},
                                ),
                                width=8,
                            )
                        )
                    )
                )
            ),
            label=f"Expansión {mode}",
        )
        tabs.append(tab)
        # Coverage
        estimate_coverage = calculate_coverage(
            worldcover, grids, start_year, 
        )
        fig_coverage = plot_coverage(estimate_coverage, f"Expansión {mode}")
        # fig_coverage.write_image(f"./test/{mode}.eps", width=1200, height=600, scale=1.5)
        coverage_graphs.append(dcc.Graph(figure=fig_coverage))

    df = pd.DataFrame(
        zip(x, y, z), columns=["Año", "Porcentaje de urbanización", "Categoría"]
    )
    fig = px.line(df, x="Año", y="Porcentaje de urbanización", color="Categoría")
    fig.update_yaxes(tickformat=",.0%")
    # fig.write_image("./test/lines.eps", width=1200, height=600, scale=1.5)

    ## Cambio Porcentual por Escenario

    base = df.loc[(df["Año"] == 2020) & (df["Categoría"] == "Observaciones")][
        "Porcentaje de urbanización"
    ].values[0]

    columns = []
    for cat, color in zip(
        ["acelerada", "inercial", "controlada"], ["danger", "warning", "success"]
    ):
        cat = f"Expansión {cat}"
        prediction = df.loc[(df["Año"] == 2070) & (df["Categoría"] == cat)][
            "Porcentaje de urbanización"
        ].values[0]
        col = dbc.Col(dbc.Card(
            dbc.CardBody(
                [
                    html.H5(cat.title(), className="card-title"),
                    html.P(
                        f"+{round(((prediction - base)/ base) * 100, 1)}% de área urbanizada 2070 vs. 2020.",
                        className="card-text",
                    ),
                ]
            ),
            color=color,
            inverse=True,
        ))
        columns.append(col)

    cards = html.Div(dbc.Row(columns, className="mb-4"))
    all_elements = [cards] + coverage_graphs + [dcc.Graph(figure=fig)]
    plot_tab = dbc.Tab(dbc.Card(dbc.CardBody(all_elements)), label="Resumen General")
    tabs = [plot_tab] + tabs
    out = dbc.Tabs(tabs, active_tab="tab-0")

    return out

In [1]:
def summary(id_hash, urban_rasters, years, language='es'):

    translations = {
    translations = {
        "es": {
            "modes": ["inercial", "acelerada", "controlada"],
            "expansion": "Expansión",
            "observations": "Observaciones",
            "urbanization_percentage": "Porcentaje de Urbanización",
            "year": "Año",
            "category": "Categoría",
            "general_summary": "Resumen General",
            "urban_area_increase": "{}% de aumento del área urbanizada 2070 vs. 2020."
        },
        "en": {
            "modes": ["inertial", "accelerated", "controlled"],
            "expansion": "Expansion",
            "observations": "Observations",
            "urbanization_percentage": "Urbanization Percentage",
            "year": "Year",
            "category": "Category",
            "general_summary": "General Summary",
            "urban_area_increase": "{}% increase in urban area 2070 vs. 2020."
        },
        "pt": {
            "modes": ["inercial", "acelerada", "controlada"],
            "expansion": "Expansão",
            "observations": "Observações",
            "urbanization_percentage": "Percentagem de Urbanização",
            "year": "Ano",
            "category": "Categoria",
            "general_summary": "Resumo Geral",
            "urban_area_increase": "{}% de aumento da área urbana 2070 vs. 2020."
        }
    }

    path_cache = Path(f"./data/cache/{id_hash}")
    worldcover = np.load(path_cache / "worldcover.npy")
    start_year = 2020
    num_years = 50
    #modes = ["inercial", "acelerada", "controlada"]
    id_hash = str(id_hash)
    historical_years = np.array(years)
    historical_grids = np.array(urban_rasters)

    x = list(historical_years)
    y = [grid.sum() / grid.size for grid in historical_grids]
    z = [translations[language]["observations"]] * len(x)

    final_y = y[-1]

    tabs = []
    coverage_graphs = []
    for mode in  for mode in translations[language]["modes"]:
        grids = load_sleuth_predictions(path_cache, id_hash, mode=mode)
        x_pred = list(range(start_year + 1, start_year + num_years + 1))
        y_pred = [grid.sum() / grid.size for grid in grids]
        z_pred = [f"{translations[language]['expansion']}: {mode}"] * (len(x_pred) + 1)
        x_pred = [start_year] + x_pred
        y_pred = [final_y] + y_pred
        x.extend(x_pred)
        y.extend(y_pred)
        z.extend(z_pred)


        tab = dbc.Tab(
            dbc.Card(
                dbc.CardBody(
                    dbc.Container(
                        dbc.Row(
                            dbc.Col(
                                dcc.Graph(
                                    figure=plot_sleuth_predictions(grids, 2020, 50),
                                    responsive=True,
                                    style={"height": "60vh"},
                                ),
                                width=8,
                            )
                        )
                    )
                )
            ),
            label=f"{translations[language]['expansion']}: {mode}",
        )
        tabs.append(tab)


        estimate_coverage = calculate_coverage(worldcover, grids, start_year)
        fig_coverage = plot_coverage(estimate_coverage, f"{translations[language]['expansion']}: {mode}")
        coverage_graphs.append(dcc.Graph(figure=fig_coverage))

    df = pd.DataFrame(
        zip(x, y, z), columns=[translations[language]["year"], translations[language]["urbanization_percentage"], translations[language]["category"]]
    )
    fig = px.line(df, x=translations[language]["year"], y=translations[language]["urbanization_percentage"], color=translations[language]["category"])
    fig.update_yaxes(tickformat=",.0%")

    base = df.loc[(df[translations[language]["year"]] == 2020) & (df[translations[language]["category"]] == translations[language]["observations"])][translations[language]["urbanization_percentage"]].values[0]

    columns = []
    for cat, color in zip(translations[language]["modes"], ["danger", "warning", "success"]):
        cat_translated = f"{translations[language]['expansion']} {cat}"
        prediction = df.loc[(df[translations[language]["year"]] == 2070) & (df[translations[language]["category"]] == cat_translated)][translations[language]["urbanization_percentage"]].values[0]
        col = dbc.Col(dbc.Card(
            dbc.CardBody(
                [
                    html.H5(cat_translated.title(), className="card-title"),
                    html.P(
                        translations[language]["urban_area_increase"].format(round(((prediction - base)/ base) * 100, 1)),
                        className="card-text",
                    ),
                ]
            ),
            color=color,
            inverse=True,
        ))
        columns.append(col)

    cards = html.Div(dbc.Row(columns, className="mb-4"))
    all_elements = [cards] + coverage_graphs + [dcc.Graph(figure=fig)]
    plot_tab = dbc.Tab(dbc.Card(dbc.CardBody(all_elements)), label=translations[language]["general_summary"])
    tabs = [plot_tab] + tabs
    out = dbc.Tabs(tabs, active_tab="tab-0")

    return out


In [ ]:
translations = {
        "es": {
            "modes": ["inercial", "acelerada", "controlada"],
            "expansion": "Expansión",
            "observations": "Observaciones",
            "urbanization_percentage": "Porcentaje de Urbanización",
            "year": "Año",
            "category": "Categoría",
            "general_summary": "Resumen General",
            "urban_area_increase": "{}% de aumento del área urbanizada 2070 vs. 2020."
        },
        "en": {
            "modes": ["inertial", "accelerated", "controlled"],
            "expansion": "Expansion",
            "observations": "Observations",
            "urbanization_percentage": "Urbanization Percentage",
            "year": "Year",
            "category": "Category",
            "general_summary": "General Summary",
            "urban_area_increase": "{}% increase in urban area 2070 vs. 2020."
        },
        "pt": {
            "modes": ["inercial", "acelerada", "controlada"],
            "expansion": "Expansão",
            "observations": "Observações",
            "urbanization_percentage": "Percentagem de Urbanização",
            "year": "Ano",
            "category": "Categoria",
            "general_summary": "Resumo Geral",
            "urban_area_increase": "{}% de aumento da área urbana 2070 vs. 2020."
        }
    }